In [30]:
#importing packages
import pandas as pd
import numpy as np
import datetime as dt

In [31]:
#reading the csv files
train = pd.read_csv("train.csv",header = None).values
test = pd.read_csv("test.csv",header = None).values
#isRetweet,retweeted, screenName,favorited, truncated, replytoSN are all irrelevant
#replyToSN, created, longitude, latitude,replytoUID is mostly NAs
#statusSource is missing from test

In [32]:
#dropping the unnecessary columns
train_sel = train[:,[1,3,5,8,12,17]]
test_sel = test[:,[1,3,5,8,11]]

In [33]:
#dropping NA values
for i in train_sel:
    for j in i:
        if j == "NA":
            j = 0
            
for i in test_sel:
    for j in i:
        if j == "NA":
            j = 0
"NA" in train_sel.any()
"NA" in test_sel.any()

False

In [38]:
#feature engineering
train_word_count = ['chars'] #converting the tweets into character counts
train_hash_count = ['#s'] #counting the number of hashtags
train_link_count = ['isLink'] #how many links are in the tweet
train_ats_count = ['atPerson'] #how many @s
train_is_retweet = ['isRT'] #was this a retweet?
for i in train_sel[1:,0]:
    train_word_count.append(len(i))
    train_hash_count.append(i.count("#"))
    train_link_count.append(i.count("https"))
    train_ats_count.append(i.count("@"))
    train_is_retweet.append("\"@" in i)

test_word_count = ['chars']
test_hash_count = ['#s']
test_link_count = ['isLink']
test_ats_count = ['atPerson']
test_is_retweet = ['isRT']

for i in test_sel[1:,0]:
    test_word_count.append(len(i))
    test_hash_count.append(i.count("#"))
    test_link_count.append(i.count("https"))
    test_ats_count.append(i.count("@"))
    test_is_retweet.append("\"@" in i)
#converting the date times
train_dt = []
test_dt = []
for i in train_sel[1:,2]:
    train_dt.append(dt.datetime.strptime(i,'%m/%d/%Y %H:%M'))
for i in test_sel[1:,2]:
    test_dt.append(dt.datetime.strptime(i,'%m/%d/%Y %H:%M'))
train_month = ['month']
train_year = ['year']
train_day = ['day']
train_time = ['time']
test_month = ['month']
test_year = ['year']
test_day = ['day']
test_time = ['time']
for i in train_dt:
    train_month.append(i.month)
    train_year.append(i.year)
    train_day.append(i.isoweekday())
    train_time.append((i.hour*60+i.minute+1800)%2400) #benchmarked time from time elapsed since 6am
for i in test_dt:
    test_month.append(i.month)
    test_year.append(i.year)
    test_day.append(i.isoweekday())
    test_time.append((i.hour*60+i.minute+1800)%2400)
train_features = [train_word_count,train_hash_count,train_link_count,train_ats_count,train_month,train_year,train_day,train_time,train_is_retweet]
test_features = [test_word_count,test_hash_count,test_link_count,test_ats_count,test_month,test_year,test_day,test_time,test_is_retweet]

In [39]:
# count number of #s
#is there hyperlink in tweet
#! points
#tweets at people
#tweets within an hour are probably the same label

In [40]:
#joining the data columns
train_ft = np.append(train_sel[:,0:5],np.zeros((train.shape[0],9)),axis = 1) #combining new feature vectors
train_ft[:,5:] = np.array([train_word_count,train_hash_count,train_link_count,train_ats_count,train_month,train_year,train_day,train_time,train_is_retweet]).T
test_ft = np.append(test_sel,np.zeros((test.shape[0],9)),axis = 1)
test_ft[:,5:] = np.array([test_word_count,test_hash_count,test_link_count,test_ats_count,test_month,test_year,test_day,test_time,test_is_retweet]).T
lbls = train_sel[:,5]
#train_ft = train_ft[]

In [43]:
#classifying the day of week to weekday or weekend
train_weekdays = np.add(np.where(train_ft[1:,11].astype(int) < 6),1)
train_weekends = np.add(np.where(train_ft[1:,11].astype(int) >= 6),1)
test_weekdays = np.add(np.where(test_ft[1:,11].astype(int) < 6),1)
test_weekends = np.add(np.where(test_ft[1:,11].astype(int) >= 6),1)

train_ft[train_weekdays,11] = 0
train_ft[train_weekends,11] = 1
test_ft[test_weekdays,11] = 0
test_ft[test_weekends,11] = 1


In [44]:
import csv #package for csv

In [45]:
n,d = train_ft.shape #writing the modified data into new csv files
with open('mod_train.csv', mode='w') as mod_train:
    mod_train = csv.writer(mod_train, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(n):
        line = train_ft[i,:]
        line = np.append(line,lbls[i])
        mod_train.writerow(line)

In [46]:
n,d = test_ft.shape
with open('mod_test.csv', mode='w') as mod_test:
    mod_test = csv.writer(mod_test, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(n):
        mod_test.writerow(test_ft[i,:])